
# Weather Feature Pipeline (Open‑Meteo) — Leaf Spot Risk (Hydrangea)

This notebook fetches **past 60 days** of hourly/daily weather from **Open‑Meteo** for a given location and photo time, then computes derived features useful for **leaf spot disease risk** (Cercospora/Septoria/Anthracnose).


In [1]:

# === 0. Requirements ===
# If running for the first time, uncomment:
!pip install requests pandas python-dateutil


In [6]:

# === 1. Parameters (to be passed from frontend or edited here) ===
from datetime import datetime, timedelta, timezone

# Example values; your frontend should pass these
PHOTO_TIMESTAMP_UTC = "2025-09-10T14:05:00Z"   # ISO string from EXIF or user
LAT = 33.948    # Athens, GA
LON = -83.377
DAYS_BACK = 60   # Open-Meteo supports longer history, but 60 days is good for v1

# If you only want to compute risk up to the day BEFORE the photo:
USE_WINDOW_END_ON_DAY_MINUS_1 = True


In [7]:

# === 2. Fetch Open-Meteo data (hourly + daily) ===
import requests
import pandas as pd

def date_range_for_photo(photo_iso: str, days_back: int = 60, day_minus_1: bool = True):
    dt = datetime.fromisoformat(photo_iso.replace("Z","")).replace(tzinfo=timezone.utc).date()
    end = (dt - timedelta(days=1)) if day_minus_1 else dt
    start = end - timedelta(days=days_back-1)
    return start.isoformat(), end.isoformat()

def fetch_open_meteo(lat: float, lon: float, start_date: str, end_date: str, timezone_str="UTC"):
    # Hourly variables for derived features
    hourly_vars = [
        "temperature_2m",
        "relative_humidity_2m",
        "precipitation",
        "cloudcover",
        "shortwave_radiation"
    ]
    # Daily variables (some are optional depending on archive)
    daily_vars = [
        "temperature_2m_max",
        "temperature_2m_min",
        "precipitation_sum",
        "et0_fao_evapotranspiration"  # may be missing depending on product; handled gracefully
    ]

    base = "https://archive-api.open-meteo.com/v1/era5"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "timezone": timezone_str,
        "hourly": ",".join(hourly_vars),
        "daily": ",".join(daily_vars)
    }
    r = requests.get(base, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()
    return data

start_date, end_date = date_range_for_photo(PHOTO_TIMESTAMP_UTC, DAYS_BACK, USE_WINDOW_END_ON_DAY_MINUS_1)
print("Query window:", start_date, "→", end_date)

data = fetch_open_meteo(LAT, LON, start_date, end_date, "UTC")

# Parse hourly
h = data.get("hourly", {})
if not h:
    raise RuntimeError("No hourly data returned from Open-Meteo.")
hourly_df = pd.DataFrame(h)
hourly_df["time"] = pd.to_datetime(hourly_df["time"])
hourly_df = hourly_df.set_index("time").sort_index()

# Parse daily
d = data.get("daily", {})
daily_df = pd.DataFrame(d)
daily_df["time"] = pd.to_datetime(daily_df["time"])
daily_df = daily_df.set_index("time").sort_index()

hourly_df.head(), daily_df.head()


Query window: 2025-07-12 → 2025-09-09


(                     temperature_2m  relative_humidity_2m  precipitation  \
 time                                                                       
 2025-07-12 00:00:00            27.0                    82            0.0   
 2025-07-12 01:00:00            25.6                    85            0.0   
 2025-07-12 02:00:00            25.2                    86            0.0   
 2025-07-12 03:00:00            24.5                    89            0.0   
 2025-07-12 04:00:00            24.1                    91            0.0   
 
                      cloudcover  shortwave_radiation  
 time                                                  
 2025-07-12 00:00:00          86                103.0  
 2025-07-12 01:00:00         100                 18.0  
 2025-07-12 02:00:00          28                  0.0  
 2025-07-12 03:00:00          57                  0.0  
 2025-07-12 04:00:00          83                  0.0  ,
             temperature_2m_max  temperature_2m_min  precipitation

In [8]:

# === 3. Derived metrics & helper functions ===
import math

def vpd_kpa(T_c, RH):
    es = 0.6108 * math.exp((17.27*T_c)/(T_c+237.3))
    ea = es * (RH/100.0)
    return max(0.0, es - ea)

def dew_point_c(T_c, RH):
    # Magnus approximation
    a, b = 17.27, 237.3
    RHc = max(1e-6, RH/100.0)
    gamma = (a*T_c)/(b+T_c) + math.log(RHc)
    return (b*gamma)/(a - gamma)

def compute_leaf_wetness_hours(hourly):
    # Proxy: hours with RH>=90 OR (T - Td)<=2°C
    count = 0
    for t, row in hourly.iterrows():
        T = float(row["temperature_2m"])
        RH = float(row["relative_humidity_2m"])
        Td = dew_point_c(T, RH)
        if (RH >= 90) or (T - Td <= 2.0):
            count += 1
    return count

def gdd_base_day(tmin, tmax, base=10.0):
    return max(0.0, ((tmin + tmax)/2.0) - base)

def rolling_features(hourly_df, daily_df, end_date):
    # Compute last 7 and 14 day windows ending at end_date (inclusive)
    d_end = pd.to_datetime(end_date)
    d7_start  = d_end - pd.Timedelta(days=6)
    d14_start = d_end - pd.Timedelta(days=13)
    
    # Slice daily
    d7  = daily_df.loc[d7_start:d_end]
    d14 = daily_df.loc[d14_start:d_end]

    # Slice hourly
    h7  = hourly_df.loc[d7_start:d_end + pd.Timedelta(days=1) - pd.Timedelta(hours=1)]
    h14 = hourly_df.loc[d14_start:d_end + pd.Timedelta(days=1) - pd.Timedelta(hours=1)]

    # Temperature
    T_mean_7 = (d7["temperature_2m_max"] + d7["temperature_2m_min"]).mean()/2.0 if len(d7) else float("nan")
    T_min_7  = d7["temperature_2m_min"].min() if len(d7) else float("nan")
    T_max_7  = d7["temperature_2m_max"].max() if len(d7) else float("nan")

    # Humidity
    RH_mean_7 = h7["relative_humidity_2m"].mean() if len(h7) else float("nan")
    RH_max_7  = h7["relative_humidity_2m"].max() if len(h7) else float("nan")

    # Rain
    Rain_sum_7  = d7["precipitation_sum"].sum() if "precipitation_sum" in d7 else float("nan")
    Rain_days_7 = (d7["precipitation_sum"] >= 1.0).sum() if "precipitation_sum" in d7 else 0
    # Rain hours from hourly precip > 0
    Rain_hours_7 = (h7["precipitation"] > 0).sum() if "precipitation" in h7 else 0

    # Leaf wetness proxy
    RH_hours_ge90_7 = (h7["relative_humidity_2m"] >= 90).sum() if "relative_humidity_2m" in h7 else 0
    LWD_hours_7 = compute_leaf_wetness_hours(h7) if len(h7) else 0

    # VPD
    VPD_mean_7 = h7.apply(lambda r: vpd_kpa(r["temperature_2m"], r["relative_humidity_2m"]), axis=1).mean() if len(h7) else float("nan")
    VPD_max_7  = h7.apply(lambda r: vpd_kpa(r["temperature_2m"], r["relative_humidity_2m"]), axis=1).max()  if len(h7) else float("nan")

    # GDD base 10C over last 14 days
    if len(d14):
        GDD_10_14 = sum(gdd_base_day(a,b,base=10.0) for a,b in zip(d14["temperature_2m_min"], d14["temperature_2m_max"]))
    else:
        GDD_10_14 = float("nan")

    # Humid streak (days with mean RH>=85)
    # We need daily mean RH; approximate from hourly by daily groupby
    daily_rh_mean = h14["relative_humidity_2m"].groupby(h14.index.date).mean() if len(h14) else pd.Series(dtype=float)
    max_streak = 0; cur = 0
    for val in daily_rh_mean.values:
        if val >= 85:
            cur += 1
            max_streak = max(max_streak, cur)
        else:
            cur = 0

    return {
        "window_end_date": str(d_end.date()),
        "T_mean_7": T_mean_7, "T_min_7": T_min_7, "T_max_7": T_max_7,
        "RH_mean_7": RH_mean_7, "RH_max_7": RH_max_7,
        "Rain_sum_7": Rain_sum_7, "Rain_days_7": int(Rain_days_7), "Rain_hours_7": int(Rain_hours_7),
        "RH_hours_ge90_7": int(RH_hours_ge90_7), "LWD_hours_7": int(LWD_hours_7),
        "VPD_mean_7": VPD_mean_7, "VPD_max_7": VPD_max_7,
        "GDD_10_14": GDD_10_14,
        "Humid_streak_10": int(max_streak)
    }


In [9]:

# === 4. Compute and print features for the last 7–14 days window ===
features = rolling_features(hourly_df, daily_df, end_date)
print("Derived weather features (ending on)", features["window_end_date"])
for k, v in features.items():
    if k != "window_end_date":
        print(f"{k}: {v}")


Derived weather features (ending on) 2025-09-09
T_mean_7: 22.82857142857143
T_min_7: 15.0
T_max_7: 31.1
RH_mean_7: 73.17857142857143
RH_max_7: 98
Rain_sum_7: 17.400000000000002
Rain_days_7: 3
Rain_hours_7: 21
RH_hours_ge90_7: 31
LWD_hours_7: 34
VPD_mean_7: 0.7891651315429797
VPD_max_7: 2.259116191701851
GDD_10_14: 174.9
Humid_streak_10: 0


In [10]:
# === Predict directly from (lat, lon, date) using your KB ===
from datetime import datetime, timedelta, timezone
import pandas as pd
import numpy as np
import json

def predict_disease_from_location_date(
    kb_path: str,
    lat: float,
    lon: float,
    date_str: str,                  # e.g., "2025-08-15"
    window_days: int = 7,           # look-back window for features (your KB rules use 7d)
    fetch_days_back: int = 21,      # how much data to fetch (>=14 because we compute some 14d feats)
    use_day_minus_1: bool = True,   # end the window on day-1 relative to the photo date
    timezone_str: str = "UTC",
    floor: float = 0.05,
    topk: int = 10
):
    # 1) Decide fetch range around the chosen date
    #    We fetch a bit more than needed so rolling_features can compute 7d & 14d windows robustly.
    dt = datetime.fromisoformat(date_str)
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    end_day = (dt.date() - timedelta(days=1)) if use_day_minus_1 else dt.date()
    start_day = end_day - timedelta(days=fetch_days_back-1)

    # 2) Fetch Open-Meteo (hourly + daily)
    data = fetch_open_meteo(lat, lon, start_day.isoformat(), end_day.isoformat(), timezone_str)

    # 3) Parse hourly/daily into DataFrames
    h = data.get("hourly", {})
    if not h:
        raise RuntimeError("No hourly data returned from Open-Meteo.")
    hourly_df = pd.DataFrame(h)
    hourly_df["time"] = pd.to_datetime(hourly_df["time"])
    hourly_df = hourly_df.set_index("time").sort_index()

    d = data.get("daily", {})
    daily_df = pd.DataFrame(d)
    daily_df["time"] = pd.to_datetime(daily_df["time"])
    daily_df = daily_df.set_index("time").sort_index()

    # 4) Compute rolling features ending at end_day
    feats = rolling_features(hourly_df, daily_df, end_day)

    # 5) Load KB and define disease order (using KB names; swap to labels.json if fusing with text model)
    kb = load_weather_kb(kb_path)
    disease_universe = [d["name"] for d in kb["diseases"]]

    # 6) Score KB → prior vector → rank
    prior = weather_prior_vector(kb, feats, disease_universe, floor=floor)
    order = np.argsort(-prior)

    # 7) Print results
    print(f"\n=== KB-based prediction for {end_day} at (lat={lat:.4f}, lon={lon:.4f}) "
          f"[window: last {window_days} days; end_on_day_minus_1={use_day_minus_1}] ===")
    for i in order[:topk]:
        print(f"{disease_universe[i]:35s}  prior={prior[i]:.3f}")

    # 8) Return handy objects
    ranked = [(disease_universe[i], float(prior[i])) for i in order]
    return ranked, feats, prior, hourly_df, daily_df

# --- Example call (Athens, GA; use your KB path below) ---
ranked, feats, prior, hdf, ddf = predict_disease_from_location_date(
    kb_path="/home/myid/bp67339/plant_disease/data/kb_leaf_spots.json",
    lat=33.948, lon=-83.377,
    date_str="2025-09-10",   # the user/photo date
    window_days=7,
    fetch_days_back=21,
    use_day_minus_1=True,
    floor=0.05,
    topk=10
)

print("\nTop-1:", ranked[0])
print("\nFeatures used (key subset):",
      {k: feats[k] for k in ["T_mean_7","RH_hours_ge90_7","LWD_hours_7","Rain_sum_7","VPD_mean_7"] if k in feats})

ValueError: could not convert string to float: '2025-09-09'

In [5]:
# ===============================
# Weather → KB: end-to-end scorer
# ===============================
import json, re, math, requests
import numpy as np
import pandas as pd
from typing import Dict, Any, List, Tuple
from datetime import datetime, timedelta, timezone
from pandas.api.types import is_datetime64_any_dtype, is_datetime64tz_dtype

# ---------- Open-Meteo fetch ----------
def fetch_open_meteo(lat: float, lon: float, start_date: str, end_date: str, timezone_str="UTC") -> Dict[str, Any]:
    hourly_vars = [
        "temperature_2m",
        "relative_humidity_2m",
        "precipitation",
        "cloudcover",
        "shortwave_radiation",
    ]
    daily_vars = [
        "temperature_2m_max",
        "temperature_2m_min",
        "precipitation_sum",
        "et0_fao_evapotranspiration",
    ]
    base = "https://archive-api.open-meteo.com/v1/era5"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "timezone": timezone_str,
        "hourly": ",".join(hourly_vars),
        "daily": ",".join(daily_vars),
    }
    r = requests.get(base, params=params, timeout=30)
    r.raise_for_status()
    return r.json()

# ---------- KB helpers (hardened) ----------
_ALLOWED = set(list("0123456789.+-*/()<>=! &|") + list("_") +
               list("abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"))

def load_weather_kb(fp: str) -> Dict[str, Any]:
    with open(fp, "r") as f:
        kb = json.load(f)
    by_name = {re.sub(r"\s+"," ",d["name"].strip().lower()): d for d in kb["diseases"]}
    kb["_by_name"] = by_name
    return kb

def _rule_true(expr: str, feats: Dict[str, Any]) -> bool:
    """Evaluate a boolean rule expression safely, substituting only numeric features."""
    s = expr.replace("&&", " and ").replace("||", " or ")
    if any(ch not in _ALLOWED for ch in s):
        return False
    for k, v in feats.items():
        # substitute only numeric values
        try:
            fv = float(v)
        except (TypeError, ValueError):
            continue
        s = re.sub(rf"\b{k}\b", str(fv), s)
    try:
        return bool(eval(s, {"__builtins__": {}}, {}))
    except Exception:
        return False

def score_weather_for_disease(d_entry: Dict[str, Any], feats: Dict[str, Any]) -> float:
    """Sum rule contributions and cap; feats can contain non-numeric keys safely."""
    sc = 0.0
    for r in d_entry.get("scoring", {}).get("rules", []):
        cond, add = r.get("if", ""), float(r.get("add", 0.0))
        if cond and _rule_true(cond, feats):
            sc += add
    cap = float(d_entry.get("scoring", {}).get("cap", 1.0))
    return max(0.0, min(cap, sc))

def weather_prior_vector(kb: Dict[str, Any],
                         feats: Dict[str, Any],
                         disease_universe: List[str],
                         floor: float = 0.05) -> np.ndarray:
    """Always sanitizes features internally; returns normalized prior over diseases."""
    out = np.zeros(len(disease_universe), dtype=np.float64)
    for i, name in enumerate(disease_universe):
        key = re.sub(r"\s+"," ",name.strip().lower())
        d_entry = kb["_by_name"].get(key)
        if d_entry is None:
            out[i] = floor
        else:
            out[i] = max(floor, float(score_weather_for_disease(d_entry, feats)))
    out /= (out.sum() + 1e-12)
    return out

# ---------- Feature engineering ----------
def vpd_kpa(T_c: float, RH: float) -> float:
    es = 0.6108 * math.exp((17.27*T_c)/(T_c+237.3))
    ea = es * (max(0.0, min(100.0, RH))/100.0)
    return max(0.0, es - ea)

def dew_point_c(T_c: float, RH: float) -> float:
    a, b = 17.27, 237.3
    RHc = max(1e-6, min(1.0, RH/100.0))
    gamma = (a*T_c)/(b+T_c) + math.log(RHc)
    return (b*gamma)/(a - gamma)

def compute_leaf_wetness_hours(hourly: pd.DataFrame) -> int:
    # Proxy: RH>=90 OR (T - Td)<=2°C
    if hourly.empty:
        return 0
    Td = hourly.apply(lambda r: dew_point_c(r["temperature_2m"], r["relative_humidity_2m"]), axis=1)
    return int(((hourly["relative_humidity_2m"] >= 90) | ((hourly["temperature_2m"] - Td) <= 2.0)).sum())

def _to_utc(ts):
    ts = pd.to_datetime(ts)
    if ts.tzinfo is None:
        return ts.tz_localize("UTC")
    return ts.tz_convert("UTC")

def rolling_features(hourly_df: pd.DataFrame, daily_df: pd.DataFrame, end_date) -> Dict[str, Any]:
    """Compute 7d/14d features ending on end_date (inclusive), with UTC-aware bounds."""
    # Make end-of-day UTC (tz-aware)
    d_end = _to_utc(end_date).normalize()  # midnight UTC of that day

    d7_start  = d_end - pd.Timedelta(days=6)
    d14_start = d_end - pd.Timedelta(days=13)

    # Ensure UTC datetime indexes
    if not (isinstance(hourly_df.index, pd.DatetimeIndex) and hourly_df.index.tz is not None):
        hf = hourly_df.copy()
        if "time" in hf.columns:
            hf["time"] = pd.to_datetime(hf["time"], utc=True)
            hourly_df = hf.set_index("time").sort_index()
        else:
            raise ValueError("hourly_df must have a tz-aware DatetimeIndex or a 'time' column.")

    if not (isinstance(daily_df.index, pd.DatetimeIndex) and daily_df.index.tz is not None):
        df = daily_df.copy()
        if "time" in df.columns:
            df["time"] = pd.to_datetime(df["time"], utc=True)
            daily_df = df.set_index("time").sort_index()
        else:
            raise ValueError("daily_df must have a tz-aware DatetimeIndex or a 'time' column.")

    # Slices (inclusive day window)
    d7  = daily_df.loc[(daily_df.index >= d7_start) & (daily_df.index <= d_end)]
    d14 = daily_df.loc[(daily_df.index >= d14_start) & (daily_df.index <= d_end)]

    h7  = hourly_df.loc[(hourly_df.index >= d7_start)  & (hourly_df.index <= d_end + pd.Timedelta(hours=23))]
    h14 = hourly_df.loc[(hourly_df.index >= d14_start) & (hourly_df.index <= d_end + pd.Timedelta(hours=23))]

    # ---- metrics (same as before) ----
    T_mean_7 = float(((d7["temperature_2m_max"] + d7["temperature_2m_min"]) / 2.0).mean()) if len(d7) else float("nan")
    T_min_7  = float(d7["temperature_2m_min"].min()) if len(d7) else float("nan")
    T_max_7  = float(d7["temperature_2m_max"].max()) if len(d7) else float("nan")

    RH_mean_7 = float(h7["relative_humidity_2m"].mean()) if len(h7) else float("nan")
    RH_max_7  = float(h7["relative_humidity_2m"].max())  if len(h7) else float("nan")

    Rain_sum_7  = float(d7["precipitation_sum"].sum()) if "precipitation_sum" in d7 else float("nan")
    Rain_days_7 = int((d7["precipitation_sum"] >= 1.0).sum()) if "precipitation_sum" in d7 else 0
    Rain_hours_7 = int((h7["precipitation"] > 0).sum()) if "precipitation" in h7 else 0

    RH_hours_ge90_7 = int((h7["relative_humidity_2m"] >= 90).sum()) if "relative_humidity_2m" in h7 else 0
    LWD_hours_7 = compute_leaf_wetness_hours(h7) if len(h7) else 0

    if len(h7):
        vpd_series = h7.apply(lambda r: vpd_kpa(r["temperature_2m"], r["relative_humidity_2m"]), axis=1)
        VPD_mean_7 = float(vpd_series.mean())
        VPD_max_7  = float(vpd_series.max())
    else:
        VPD_mean_7 = float("nan"); VPD_max_7 = float("nan")

    if len(h14):
        daily_rh_mean = h14["relative_humidity_2m"].groupby(h14.index.date).mean()
        max_streak = 0; cur = 0
        for val in daily_rh_mean.values:
            if val >= 85:
                cur += 1; max_streak = max(max_streak, cur)
            else:
                cur = 0
    else:
        max_streak = 0

    return {
        "window_end_date": str(d_end.date()),
        "T_mean_7": T_mean_7, "T_min_7": T_min_7, "T_max_7": T_max_7,
        "RH_mean_7": RH_mean_7, "RH_max_7": RH_max_7,
        "Rain_sum_7": Rain_sum_7, "Rain_days_7": Rain_days_7, "Rain_hours_7": Rain_hours_7,
        "RH_hours_ge90_7": RH_hours_ge90_7, "LWD_hours_7": LWD_hours_7,
        "VPD_mean_7": VPD_mean_7, "VPD_max_7": VPD_max_7,
        "Humid_streak_10": int(max_streak),
    }
# ---------- Main prediction APIs ----------
def predict_disease_from_location_date(
    kb_path: str,
    lat: float,
    lon: float,
    date_str: str,                 # "YYYY-MM-DD"
    fetch_days_back: int = 21,     # >=14 for 14d feats
    use_day_minus_1: bool = True,  # end window on previous day
    timezone_str: str = "UTC",
    floor: float = 0.05,
    topk: int = 10
) -> Tuple[List[Tuple[str, float]], Dict[str, Any], np.ndarray, pd.DataFrame, pd.DataFrame]:
    # resolve date range
    dt = datetime.fromisoformat(date_str)
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    end_day = (dt.date() - timedelta(days=1)) if use_day_minus_1 else dt.date()
    start_day = end_day - timedelta(days=fetch_days_back-1)

    # fetch weather
    data = fetch_open_meteo(lat, lon, start_day.isoformat(), end_day.isoformat(), timezone_str)

    # parse hourly/daily
    h = data.get("hourly", {})
    if not h:
        raise RuntimeError("No hourly data returned from Open-Meteo.")
    hourly_df = pd.DataFrame(h)
    hourly_df["time"] = pd.to_datetime(hourly_df["time"], utc=True)
    hourly_df = hourly_df.set_index("time").sort_index()

    d = data.get("daily", {})
    daily_df = pd.DataFrame(d)
    daily_df["time"] = pd.to_datetime(daily_df["time"], utc=True)
    daily_df = daily_df.set_index("time").sort_index()

    # features over last 7 days ending at end_day
    feats = rolling_features(hourly_df, daily_df, end_day)

    # KB scoring
    kb = load_weather_kb(kb_path)
    disease_universe = [d["name"] for d in kb["diseases"]]
    prior = weather_prior_vector(kb, feats, disease_universe, floor=floor)
    order = np.argsort(-prior)

    # print
    print(f"\n=== KB-based prediction for {end_day} at (lat={lat:.4f}, lon={lon:.4f}) "
          f"[window: last 7 days; end_on_day_minus_1={use_day_minus_1}] ===")
    for i in order[:topk]:
        print(f"{disease_universe[i]:35s}  prior={prior[i]:.3f}")

    ranked = [(disease_universe[i], float(prior[i])) for i in order]
    return ranked, feats, prior, hourly_df, daily_df

def predict_disease_from_weather_df(
    kb_path: str,
    hourly_df: pd.DataFrame,
    daily_df: pd.DataFrame,
    end_day,                     # date or Timestamp
    floor: float = 0.05,
    topk: int = 10
) -> Tuple[List[Tuple[str, float]], Dict[str, Any], np.ndarray]:
    """Use this if you already have local weather DataFrames."""
    # ensure indices are datetime and UTC
    def _ensure_dtidx(df, colname="time"):
        if isinstance(df.index, pd.RangeIndex) or not (is_datetime64_any_dtype(df.index) or is_datetime64tz_dtype(df.index)):
            if colname in df.columns:
                df = df.copy()
                df[colname] = pd.to_datetime(df[colname], utc=True)
                df = df.set_index(colname)
            else:
                raise ValueError("DataFrame must have a datetime index or a datetime column named 'time'.")
        return df.sort_index()

    hourly_df = _ensure_dtidx(hourly_df)
    daily_df  = _ensure_dtidx(daily_df)

    feats = rolling_features(hourly_df, daily_df, end_day)
    kb = load_weather_kb(kb_path)
    disease_universe = [d["name"] for d in kb["diseases"]]
    prior = weather_prior_vector(kb, feats, disease_universe, floor=floor)
    order = np.argsort(-prior)

    print(f"\n=== KB-based prediction for {pd.to_datetime(end_day).date()} (preloaded DF) ===")
    for i in order[:topk]:
        print(f"{disease_universe[i]:35s}  prior={prior[i]:.3f}")

    ranked = [(disease_universe[i], float(prior[i])) for i in order]
    return ranked, feats, prior


ranked, feats, prior, hdf, ddf = predict_disease_from_location_date(
    kb_path="/home/myid/bp67339/plant_disease/data/kb_leaf_spots.json",
    lat=33.948, lon=-83.377,
    date_str="2025-09-17",   # photo date
    fetch_days_back=21,
    use_day_minus_1=True,
    floor=0.05,
    topk=10
)
print("\nTop-1:", ranked[0])
print("\nFeature snapshot:",
      {k: feats[k] for k in ["T_mean_7","RH_hours_ge90_7","LWD_hours_7","Rain_sum_7","VPD_mean_7"] if k in feats})


=== KB-based prediction for 2025-09-16 at (lat=33.9480, lon=-83.3770) [window: last 7 days; end_on_day_minus_1=True] ===
Spot Anthracnose (Dogwood)           prior=0.167
Phyllosticta Leaf Spot (ornamental shrubs incl. Hydrangea)  prior=0.167
Bacterial Leaf Spot (Hydrangea)      prior=0.143
Septoria Leaf Spot                   prior=0.143
Bacterial Leaf Scorch (physiological look-alike)  prior=0.143
Cercospora Leaf Spot                 prior=0.119
Anthracnose (leaf spot/blight on Hydrangea)  prior=0.119

Top-1: ('Spot Anthracnose (Dogwood)', 0.16666666666658728)

Feature snapshot: {'T_mean_7': 21.866666666666664, 'RH_hours_ge90_7': 5, 'LWD_hours_7': 8, 'Rain_sum_7': 0.30000000000000004, 'VPD_mean_7': 0.8416018131723381}


In [6]:
# Pretty table
import pandas as pd, json, os

feats_df = pd.DataFrame([feats]).T.rename(columns={0: "value"})
feats_df.index.name = "feature"
print("\n=== All derived weather features (table) ===")
display(feats_df)  # in Jupyter shows a clean table

# Optional: save for your fusion notebook
out_fp = "/home/myid/bp67339/plant_disease/data/last_weather_feats.json"
with open(out_fp, "w") as f:
    json.dump(feats, f, indent=2)
print(f"\nSaved features to: {out_fp}")


=== All derived weather features (table) ===


,value
feature,
window_end_date,2025-09-16
T_mean_7,21.866667
T_min_7,14.2
T_max_7,28.4
RH_mean_7,67.055172
RH_max_7,91.0
Rain_sum_7,0.3
Rain_days_7,0
Rain_hours_7,3



Saved features to: /home/myid/bp67339/plant_disease/data/last_weather_feats.json


In [1]:
# Past-1-month weather table (temp, humidity, rainfall)
# ----------------------------------------------------
# Requirements: requests, pandas
# pip install requests pandas

import math
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

def _to_iso_date(d):
    if isinstance(d, str):
        return d
    if isinstance(d, datetime):
        return d.date().isoformat()
    return pd.to_datetime(d).date().isoformat()

def vpd_kpa(T_c: float, RH: float) -> float:
    # Not used in the table, but handy if you want VPD later
    es = 0.6108 * math.exp((17.27*T_c)/(T_c+237.3))
    ea = es * (max(0.0, min(100.0, RH))/100.0)
    return max(0.0, es - ea)

def fetch_month_weather_table(
    lat: float,
    lon: float,
    end_date: datetime | str | None = None,
    days: int = 30,
    timezone_str: str = "UTC",
) -> pd.DataFrame:
    """
    Returns a tidy daily table for the past `days` ending at `end_date`
    with columns: Date, T_min (°C), T_max (°C), T_mean (°C), RH_mean (%), Rain_sum (mm).
    """
    # Resolve dates
    end_dt = datetime.now(timezone.utc) if end_date is None else pd.to_datetime(end_date, utc=True).to_pydatetime()
    end_day = end_dt.date()
    start_day = end_day - timedelta(days=days-1)

    # Open-Meteo ERA5 archive
    base = "https://archive-api.open-meteo.com/v1/era5"
    hourly_vars = ["temperature_2m", "relative_humidity_2m", "precipitation"]
    daily_vars  = ["temperature_2m_max", "temperature_2m_min", "precipitation_sum"]

    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_day.isoformat(),
        "end_date": end_day.isoformat(),
        "timezone": timezone_str,
        "hourly": ",".join(hourly_vars),
        "daily": ",".join(daily_vars),
    }

    r = requests.get(base, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()

    # Daily frame (min/max temp, daily rain)
    d = pd.DataFrame(data.get("daily", {}))
    if d.empty:
        raise RuntimeError("No daily data returned.")
    d["time"] = pd.to_datetime(d["time"])
    d = d.set_index("time").sort_index()

    # Hourly to compute daily RH mean (and optional T mean)
    h = pd.DataFrame(data.get("hourly", {}))
    if h.empty:
        raise RuntimeError("No hourly data returned.")
    h["time"] = pd.to_datetime(h["time"])
    h = h.set_index("time").sort_index()

    # Aggregate hourly → daily means
    rh_daily_mean = h["relative_humidity_2m"].groupby(h.index.date).mean()
    t_daily_mean  = h["temperature_2m"].groupby(h.index.date).mean()

    # Assemble final table
    out = pd.DataFrame(index=d.index)
    out.index.name = "Date"

    out["T_min (°C)"]  = d["temperature_2m_min"]
    out["T_max (°C)"]  = d["temperature_2m_max"]
    out["T_mean (°C)"] = pd.Series(t_daily_mean.values, index=pd.to_datetime(rh_daily_mean.index))
    out["RH_mean (%)"] = pd.Series(rh_daily_mean.values, index=pd.to_datetime(rh_daily_mean.index))
    out["Rain_sum (mm)"] = d["precipitation_sum"]

    # Tidy up: round and sort columns
    out = out[["T_min (°C)", "T_max (°C)", "T_mean (°C)", "RH_mean (%)", "Rain_sum (mm)"]]
    out = out.round({"T_min (°C)": 1, "T_max (°C)": 1, "T_mean (°C)": 1, "RH_mean (%)": 1, "Rain_sum (mm)": 2})
    return out

# ---- Example usage ----
if __name__ == "__main__":
    # Example: Athens, GA
    lat, lon = 33.948, -83.377
    table = fetch_month_weather_table(lat, lon, timezone_str="America/New_York")

    # Display (in notebooks, this renders nicely)
    print(table.to_string())

    # Save to CSV if you want
    table.to_csv("last_30_days_weather.csv")
    print("\nSaved to last_30_days_weather.csv")

            T_min (°C)  T_max (°C)  T_mean (°C)  RH_mean (%)  Rain_sum (mm)
Date                                                                       
2025-08-19        22.7        28.4         25.4         86.9            4.8
2025-08-20        21.9        30.3         25.6         84.0            0.1
2025-08-21        21.3        30.9         25.1         85.6            8.0
2025-08-22        21.5        27.4         23.2         92.6            8.8
2025-08-23        20.4        25.4         22.5         92.1            4.0
2025-08-24        18.8        29.2         23.5         81.9            0.3
2025-08-25        20.2        29.1         24.3         67.3            0.0
2025-08-26        16.4        27.0         21.9         60.8            0.0
2025-08-27        16.0        26.8         21.1         61.0            0.0
2025-08-28        16.7        27.2         21.8         68.6            0.0
2025-08-29        16.7        28.1         22.5         72.5            0.0
2025-08-30  

In [1]:
# Core
import os, json, math, datetime as dt
from typing import Dict, Any, List

# HTTP
import requests

# Data
import pandas as pd

# Display options
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)

# Location & horizon
LAT = 33.95      # <- change to your latitude
LON = -83.37     # <- change to your longitude
TZ  = "UTC"      # e.g., "America/New_York" if you prefer local time
DAYS_AHEAD = 16  # Open-Meteo forecast horizon (max ~16 days)

# Date window: today -> today+DAYS_AHEAD
today = dt.date.today()
start_date = today.isoformat()
end_date   = (today + dt.timedelta(days=DAYS_AHEAD)).isoformat()

print("Start:", start_date, "| End:", end_date, "| TZ:", TZ)

Start: 2025-09-22 | End: 2025-10-08 | TZ: UTC


In [2]:
# Pick only what you need. You can add/remove variables from:
# https://open-meteo.com/ (Docs → Forecast API variables)

hourly_vars = [
    "temperature_2m",
    "relative_humidity_2m",
    "precipitation",
    "cloudcover",
    "wind_speed_10m",
    "wind_gusts_10m",
]

daily_vars = [
    "temperature_2m_max",
    "temperature_2m_min",
    "precipitation_sum",
    "sunrise",
    "sunset",
]

In [5]:
def fetch_open_meteo_forecast(
    lat, lon, timezone_str, hourly, daily, forecast_days=16, timeout=30
):
    base = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": lat,
        "longitude": lon,
        "timezone": timezone_str,
        "forecast_days": int(forecast_days),   # <= use this
        "past_days": 0,                        # (optional) ensure only future
        "hourly": ",".join(hourly) if hourly else None,
        "daily": ",".join(daily) if daily else None,
    }
    params = {k: v for k, v in params.items() if v is not None}
    r = requests.get(base, params=params, timeout=timeout)
    try:
        r.raise_for_status()
    except requests.HTTPError as e:
        # Helpful debug
        raise RuntimeError(f"Open-Meteo error {r.status_code}: {r.text}") from e
    return r.json()

In [7]:
raw = fetch_open_meteo_forecast(
    LAT, LON, TZ, hourly=hourly_vars, daily=daily_vars, forecast_days=16
)

# Hourly table
hourly = pd.DataFrame(raw.get("hourly", {}))
if not hourly.empty:
    hourly["time"] = pd.to_datetime(hourly["time"])
    hourly = hourly.set_index("time").sort_index()

# Daily table
daily = pd.DataFrame(raw.get("daily", {}))
if not daily.empty:
    daily["time"] = pd.to_datetime(daily["time"])
    daily = daily.set_index("time").sort_index()

print("=== Hourly Forecast (first 24 hours) ===")
print(hourly.head(24))

print("\n=== Daily Forecast (full 16 days) ===")
print(daily)

=== Hourly Forecast (first 24 hours) ===
                     temperature_2m  relative_humidity_2m  precipitation  cloudcover  wind_speed_10m  wind_gusts_10m
time                                                                                                                
2025-09-22 00:00:00            23.0                    80            0.0           6             5.6            10.8
2025-09-22 01:00:00            21.4                    88            0.0           0             6.1            10.8
2025-09-22 02:00:00            20.5                    85            0.0          50             1.8            12.2
2025-09-22 03:00:00            20.0                    88            0.0          11             3.2             5.0
2025-09-22 04:00:00            19.9                    89            0.0          13             3.3             5.8
2025-09-22 05:00:00            19.5                    84            0.0           3             7.9            20.2
2025-09-22 06:00:00    